# Top Campaign donors in Washington State

This notebook analyzes bulk data retrieved from [Open Secrets](http://www.opensecrets.org/myos/bulk.php) - specifically, donations to federal campaigns for the 2016 cycle through most of January.

Let's load the basic data and see what we've got.

In [1]:
import agate

table = agate.Table.from_csv('indivs_Washington16.csv')
print [c.name for c in table.columns]

[u'Cycle', u'Fectransid', u'Contribid', u'Contrib', u'Recipid', u'Orgname', u'Ultorg', u'Realcode', u'Date', u'Amount', u'Street', u'City', u'State', u'Zip', u'Recipcode', u'Type', u'CmteId', u'OtherID', u'Gender', u'Microfilm', u'Occupation', u'Employer', u'Source']


Next, let's make a pivot table off of it - just contributor name, their organization, their zip code, and how much they've given (taking into account that sum entries in the data account for refunds, i.e. negative amounts).

In [2]:
pivot = table.pivot(['Contrib', 'Orgname', 'Zip'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)

In [3]:
for row in pivot.order_by('Sum', reverse=True).limit(10).rows:
    print row['Contrib'], row['Orgname'], row['Zip'], row['Sum']

BALLMER, CONNIE E Partners for Our Children 98004 250000
PUYALLUP TRIBE OF INDIANS Puyallup Tribe of Indians 98404 186100
SCHWEITZER, BEATRIZ VALDEZ MRS Schweitzer Engineering Labs 99163 173000
SCHWEITZER, EDMUND O DR Schweitzer Engineering Labs 99163 173000
SCHWEITZER, BEATRIZ V Schweitzer Engineering Labs 99163 155000
SCHWEITZER, III, EDMUND O Schweitzer Engineering Labs 99163 150000
None Homemaker 99163 106200
None SCHWEITZER ENGINEERING LABS 99163 106200
SNOQUALMIE TRIBE Snoqualmie Tribe 98065 104800
MUCKLESHOOT INDIAN TRIBE Muckleshoot Indian Tribe 98092 75700


Now we need to be able to see what the contributor IDs mean, so we'll ask ProPublica via [their API](https://propublica.github.io/campaign-finance-api-docs).

In [4]:
import json
import os
import pycurl
from StringIO import StringIO

def get_cmte_from_id(cmte_id):
    api_key = os.getenv('PROPUBLICA_API_KEY')
    url = 'https://api.propublica.org/campaign-finance/v1/2016/committees/%s.json' % cmte_id

    buffer = StringIO()
    c = pycurl.Curl()
    c.setopt(c.URL, url)
    c.setopt(c.HTTPHEADER, ['X-API-KEY: %s' % api_key])
    c.setopt(c.WRITEDATA, buffer)
    c.perform()
    c.close()

    body = buffer.getvalue()
    # Body is a string in some encoding.
    # In Python 2, we can print it without knowing what the encoding is.
    try:
        return json.loads(body)['results'][0]
    except:
        return {}

Save this mapping to a file so we don't have to rebuild it later.

In [5]:
mapping = {}

for cmte in table.columns['CmteId'].values_distinct():
    mapping[cmte] = get_cmte_from_id(cmte)

print len(mapping)

776


In [6]:
with open('mapping.json', 'w+') as fh:
    fh.write(json.dumps(mapping))

Now annotate the table (and save it to `contribs_with_cmtes` when we're done).

In [7]:
with_cmtes = table.compute([
    ('CmteName', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['name'])),
    ('CmteParty', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['party'])),
    ('CmteAddress', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['address'])),
    ('CmteCity', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['city'])),
    ('CmteState', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['state'])),
    ('CmteZip', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['zip'])),
    ('CmteType', agate.Formula(agate.Text(), lambda k: mapping[k['CmteId']]['committee_type'])),
])

In [8]:
with_cmtes.to_csv('contribs_with_cmtes.csv')

What committees raised the most money from Washington?

In [9]:
top_cmtes_pivot = with_cmtes.pivot(['CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)

In [10]:
for row in top_cmtes_pivot.order_by('Sum', reverse=True).limit(10).rows:
    print '%s raised %d' % (row['CmteName'], row['Sum'])

PEOPLE FOR PATTY MURRAY raised 983429
HILLARY FOR AMERICA raised 860582
NRCC raised 763246
DSCC raised 572685
MURRAY VICTORY FUND raised 498006
ACTBLUE raised 414294
REPUBLICAN NATIONAL COMMITTEE raised 385715
BERNIE 2016 raised 384694
PEOPLE FOR DEREK KILMER raised 354832
WASHINGTON STATE REPUBLICAN PARTY raised 340495


Let's break out just presidential committees.

In [11]:
pres_only = with_cmtes.where(lambda k: k['CmteType'] == 'P')

Who were the top donors to presidential committees? (Save the full table to `top_pres_donors.csv`)

In [12]:
top_pres_donors_pivot = pres_only.pivot(
    ['Contrib', 'CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
top_pres_donors_pivot.limit(10).print_table()
top_pres_donors_pivot.to_csv('top_pres_donors.csv')

|-----------------------+----------------------+---------|
|  Contrib              | CmteName             |    Sum  |
|-----------------------+----------------------+---------|
|  GAMORAN, SAUL        | CRUZ FOR PRESIDENT   | 10,800  |
|  MEISENBACH, JOHN     | CRUZ FOR PRESIDENT   | 10,000  |
|  LANDON, JOSEPH PA... | JEB 2016, INC.       |  8,100  |
|  TUCKER, ROBERT       | JEB 2016, INC.       |  8,100  |
|  ERWIN, GERALD        | CARSON AMERICA       |  7,700  |
|  NEUPERT, SHERYL S... | MARCO RUBIO FOR P... |  5,400  |
|  JULIAN, REBECCA L    | CRUZ FOR PRESIDENT   |  5,400  |
|  RICE, MERIDEL        | CRUZ FOR PRESIDENT   |  5,400  |
|  SABOL, JOHN          | HILLARY FOR AMERICA  |  5,400  |
|  KETCHAM, SAM         | HILLARY FOR AMERICA  |  5,400  |
|-----------------------+----------------------+---------|


What were the top parties by fundraising, and how much did they get (and from whom)? (Save the full tables to `top_parties.csv` and `top_donor_parties.csv`)

In [18]:
top_parties = with_cmtes.pivot(['CmteParty'], aggregation=agate.Sum('Amount')).order_by('Sum',reverse=True)
top_parties.limit(10).print_table()
top_parties.to_csv('top_parties.csv')

|------------+------------|
|  CmteParty |       Sum  |
|------------+------------|
|            | 5,692,250  |
|  DEM       | 4,492,105  |
|  REP       | 3,779,814  |
|  UNK       |   380,854  |
|  NNE       |    87,011  |
|  IND       |    22,100  |
|  DFL       |     8,900  |
|  GRE       |     6,850  |
|  LIB       |     5,005  |
|  WOR       |       500  |
|------------+------------|


In [19]:
top_donor_parties = with_cmtes.pivot(
    ['Contrib', 'CmteParty'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
top_donor_parties.limit(10).print_table()
top_donor_parties.to_csv('top_donor_parties.csv')

|-----------------------+-----------+----------|
|  Contrib              | CmteParty |     Sum  |
|-----------------------+-----------+----------|
|  BALLMER, CONNIE E    |           | 250,000  |
|                       | REP       | 212,400  |
|  SCHWEITZER, BEATR... | REP       | 173,000  |
|  SCHWEITZER, EDMUN... | REP       | 173,000  |
|  SCHWEITZER, BEATR... |           | 155,000  |
|  SCHWEITZER, III, ... |           | 150,000  |
|  PUYALLUP TRIBE OF... | DEM       | 130,500  |
|  KEITH, MICHAEL       | DEM       |  93,405  |
|  CLARK, JOSEPH        | REP       |  72,000  |
|  MUCKLESHOOT INDIA... | DEM       |  70,500  |
|-----------------------+-----------+----------|


What were the largest donor-committee pairs? I.e., which donors gave the most to whom? (Save the full table to `top_cmte_donors.csv`)

In [14]:
top_cmte_donors = with_cmtes.pivot(
    ['Contrib', 'CmteName', 'CmteParty'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
top_cmte_donors.limit(10).print_table()
top_cmte_donors.to_csv('top_cmte_donors.csv')

|-----------------------+----------------------+-----------+----------|
|  Contrib              | CmteName             | CmteParty |     Sum  |
|-----------------------+----------------------+-----------+----------|
|  BALLMER, CONNIE E    | AMERICANS FOR RES... |           | 250,000  |
|                       | NRCC                 | REP       | 212,400  |
|  SCHWEITZER, BEATR... | MCMORRIS RODGERS ... |           | 150,000  |
|  SCHWEITZER, III, ... | MCMORRIS RODGERS ... |           | 150,000  |
|  SCHWEITZER, BEATR... | NRCC                 | REP       | 139,600  |
|  SCHWEITZER, EDMUN... | NRCC                 | REP       | 139,600  |
|  CLARK, JOSEPH        | REPUBLICAN NATION... | REP       |  66,600  |
|  PUYALLUP TRIBE OF... | DSCC                 | DEM       |  64,800  |
|  MCCAW, CRAIG MR      | RIGHT TO RISE USA    |           |  50,000  |
|  LANDON, JOSEPH MR    | RIGHT TO RISE USA    |           |  50,000  |
|-----------------------+----------------------+-----------+----

Let's rank all the zipcodes in our data by how much they've given (so we can do a choropleth later, if we want). Save the full table to `zip_amounts.csv`.

In [22]:
zip_fives = with_cmtes.compute([
    ('Zip5', agate.Formula(agate.Text(), lambda k: str(k['Zip'])[:5]))
])

In [24]:
zip_five_pivot = zip_fives.pivot(['Zip5'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
zip_five_pivot.limit(10).print_table()
zip_five_pivot.to_csv('zip_amounts.csv')

|--------+----------|
|  Zip5  |     Sum  |
|--------+----------|
|  98004 | 967,854  |
|  99163 | 895,368  |
|  98101 | 473,987  |
|  98040 | 468,684  |
|  98112 | 423,031  |
|  98039 | 339,659  |
|  98119 | 332,749  |
|  98177 | 285,893  |
|  98105 | 272,272  |
|  98033 | 256,558  |
|--------+----------|


Can we have that pivot table group by committee too? (`zip_cmte_amounts.csv`)

In [25]:
zip_five_committee_pivot = zip_fives.pivot(
    ['Zip5', 'CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
zip_five_committee_pivot.limit(10).print_table()
zip_five_committee_pivot.to_csv('zip_cmte_amounts.csv')

|--------+----------------------+----------|
|  Zip5  | CmteName             |     Sum  |
|--------+----------------------+----------|
|  99163 | NRCC                 | 491,600  |
|  99163 | MCMORRIS RODGERS ... | 300,000  |
|  98004 | AMERICANS FOR RES... | 250,000  |
|  98108 | REPUBLICAN NATION... | 100,000  |
|  98112 | HILLARY FOR AMERICA  |  78,600  |
|  98004 | HILLARY FOR AMERICA  |  75,050  |
|  98177 | DSCC                 |  71,550  |
|  98083 | REPUBLICAN NATION... |  66,800  |
|  99163 | NRSC                 |  66,800  |
|  98070 | DSCC                 |  66,518  |
|--------+----------------------+----------|


What are Hillary's top zips? How about Bernie's? Trump's? Cruz's?

In [34]:
for cmte in [
        'Hillary for America',
        'Bernie 2016',
        'Donald J. Trump for President, Inc.',
        'Cruz for President']:
    cmte_zip_pivot = zip_fives.where(lambda k: k['CmteName'] == cmte.upper()).pivot(
        ['Zip5'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
    print cmte
    cmte_zip_pivot.limit(10).print_table()
    cmte_zip_pivot.to_csv('%s_zip_amounts.csv' % cmte.replace(' ', '').replace('.', '').replace(',', '').lower())

Hillary for America
|--------+---------|
|  Zip5  |    Sum  |
|--------+---------|
|  98112 | 78,600  |
|  98004 | 75,050  |
|  98119 | 44,056  |
|  98144 | 43,935  |
|  98102 | 35,750  |
|  98040 | 34,600  |
|  98109 | 27,100  |
|  98101 | 27,010  |
|  98117 | 20,563  |
|  98121 | 19,500  |
|--------+---------|
Bernie 2016
|--------+---------|
|  Zip5  |    Sum  |
|--------+---------|
|  98122 | 13,250  |
|  98112 | 12,021  |
|  98070 | 11,480  |
|  98110 | 10,921  |
|  98115 | 10,149  |
|  98117 |  9,650  |
|  98103 |  8,800  |
|  98119 |  8,350  |
|  98144 |  7,550  |
|  98105 |  7,216  |
|--------+---------|
Donald J. Trump for President, Inc.
|--------+--------|
|  Zip5  |   Sum  |
|--------+--------|
|  98020 | 2,950  |
|  98004 | 2,700  |
|  98248 | 2,700  |
|  98039 | 2,700  |
|  98331 | 1,120  |
|  98390 | 1,000  |
|  98312 | 1,000  |
|  98112 | 1,000  |
|  99217 |   500  |
|  98363 |   500  |
|--------+--------|
Cruz for President
|--------+---------|
|  Zip5  |    Sum  |
|--

What are Hillary's top employers? Occupations? Same for the others?

In [33]:
for cmte in [
        ['Hillary for America'],
        ['Bernie 2016', 'Friends of Bernie Sanders'],
        ['Donald J. Trump for President, Inc.'],
        ['Cruz for President']]:
    print '; '.join(cmte)
    for field in ['Employer', 'Occupation']:
        cmte_pivot = with_cmtes.where(lambda k: k['CmteName'] in [c.upper() for c in cmte]).pivot(
            [field], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
        print field
        cmte_pivot.limit(10).print_table()
        cmte_pivot.to_csv('%s_%s_amounts.csv' % (
                cmte[0].replace(' ', '').replace('.', '').replace(',', '').lower(), field.lower()))

Hillary for America
Employer
|-----------------------+----------|
|  Employer             |     Sum  |
|-----------------------+----------|
|                       | 186,267  |
|  SELF-EMPLOYED        | 151,611  |
|  RETIRED              |  35,399  |
|  MICROSOFT CORPORA... |  31,885  |
|  UNIVERSITY OF WAS... |  21,700  |
|  STARBUCKS COFFEE ... |  21,500  |
|  STARBUCKS            |   8,850  |
|  GMMB                 |   7,900  |
|  INTELLECTUAL VENT... |   6,150  |
|  DLA PIPER LLP        |   5,400  |
|-----------------------+----------|
Occupation
|-------------+----------|
|  Occupation |     Sum  |
|-------------+----------|
|  RETIRED    | 110,372  |
|  ATTORNEY   |  42,497  |
|             |  38,895  |
|  CONSULTANT |  29,150  |
|  LAWYER     |  25,850  |
|  EXECUTIVE  |  23,450  |
|  HOMEMAKER  |  23,100  |
|  STUDENT    |  21,600  |
|  PROFESSOR  |  17,250  |
|  PHYSICIAN  |  17,250  |
|-------------+----------|
Bernie 2016; Friends of Bernie Sanders
Employer
|---------------

What have each of the Presidential committees raised?

In [30]:
pres_cmtes_pivot = with_cmtes.where(lambda k: k['CmteType'] == 'P').pivot(
    ['CmteName'], aggregation=agate.Sum('Amount')).order_by('Sum', reverse=True)
pres_cmtes_pivot.limit(20).print_table()
pres_cmtes_pivot.to_csv('pres_cmte_amounts.csv')

|-----------------------+----------|
|  CmteName             |     Sum  |
|-----------------------+----------|
|  HILLARY FOR AMERICA  | 860,582  |
|  BERNIE 2016          | 384,694  |
|  JEB 2016, INC.       | 225,750  |
|  CARSON AMERICA       | 180,917  |
|  CRUZ FOR PRESIDENT   | 148,723  |
|  MARCO RUBIO FOR P... | 148,150  |
|  RAND PAUL FOR PRE... |  80,978  |
|  CARLY FOR PRESIDENT  |  70,728  |
|  SCOTT WALKER INC     |  49,608  |
|  DONALD J. TRUMP F... |  19,084  |
|  KASICH FOR AMERICA   |  16,250  |
|  WEBB 2016            |  13,900  |
|  LINDSEY GRAHAM 2016  |  11,450  |
|  O'MALLEY FOR PRES... |   7,000  |
|  PERRY FOR PRESIDE... |   6,900  |
|  JILL STEIN FOR PR... |   5,100  |
|  JINDAL FOR PRESIDENT |   4,750  |
|  HUCKABEE FOR PRES... |   3,950  |
|  CHRIS CHRISTIE FO... |   3,200  |
|  GILMORE FOR AMERI... |   2,700  |
|-----------------------+----------|
